In [18]:
import time
import astroquery.heasarc
from astroquery.simbad import Simbad
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from collections import defaultdict
from astropy.time import Time

Idea of how this code works

1. We obtain Jupiter's position in the sky (ephemerides). We will be using a search radius of 8 degrees (due to FOV of INTEGRAL, fully coded). I found that Jupiter's position does not significantly change (more than 8 degrees) within less than 2 months. For safety, we will obtain Jupiter's mean monthly position.
2. Using the mean monthly position of Jupiter over a certain time period (1 year, 2 years, ...), we will search for all the ScWs within 8 degrees of this position. No time filtering yet. We will group by mean monthly position of Jupiter.
Though we already filter for pointing ScWs only.
3. Then, we have N groups of ScWs, each corresponding to a certain RA and DEC (within 8 degrees). These only contain Jupiter if it is in the FOV at that precise time, i.e. that month. So filtering will be done by taking ScWs only within corresponding month (aka the month corresponding to the mean position as found in step 1).
4. Finally, we will have N groups of ScWs, each containg Jupiter, for each month in the given interval. Some may be empty.

STEP 1 - obtaining Jupiter's position

In [2]:
# First, define the time interval we are interested in (> 1 month)
epochs = {'start':'2003-01-01', 'stop':'2025-01-01', 'step':'31d'}

# Get Jupiter's monthly position in the given time interval (not using mean as step of 31 days)
jupiter = Horizons(id='599', location='@0',epochs=epochs) 
eph = jupiter.ephemerides()

ra_values = eph['RA']
dec_values = eph['DEC']
datetime_values = eph['datetime_jd']
print(f"Jupiter's monthly position between {epochs['start']} and {epochs['stop']}: \n")
print(eph[['datetime_str', 'RA', 'DEC']])

monthly_intervals = []
for i in range(len(datetime_values) - 1):  # Get the intervals between successive months
    start_time = Time(datetime_values[i], format='jd').mjd
    end_time = Time(datetime_values[i + 1], format='jd').mjd
    monthly_intervals.append((start_time, end_time))
print(f"Monthly intervals based on Jupiter's positions:\n {monthly_intervals}")

# If we want to use mean:
# df = eph.to_pandas()
# df['datetime_str'] = pd.to_datetime(df['datetime_str'])
# monthly_means = df.groupby(df['datetime_str'].dt.strftime('%Y-%m'))[['RA', 'DEC']].mean()
# for month, row in monthly_means.iterrows():
    # print(f"Month: {month}, Mean RA: {row['RA']:.2f} deg, Mean DEC: {row['DEC']:.2f} deg")

Jupiter's monthly position between 2003-01-01 and 2025-01-01: 

   datetime_str       RA      DEC   
       ---           deg      deg   
----------------- --------- --------
2003-Jan-01 00:00 133.13461 18.23402
2003-Feb-01 00:00 135.64353 17.59249
2003-Mar-04 00:00  138.1248 16.92216
2003-Apr-04 00:00 140.57899 16.22478
2003-May-05 00:00 143.00678 15.50206
2003-Jun-05 00:00 145.40891  14.7557
2003-Jul-06 00:00 147.78632 13.98736
2003-Aug-06 00:00    150.14 13.19866
2003-Sep-06 00:00 152.47101 12.39117
2003-Oct-07 00:00 154.78053 11.56645
              ...       ...      ...
2024-Mar-21 00:00  50.60927 17.52697
2024-Apr-21 00:00  53.43087 18.24997
2024-May-22 00:00  56.26678 18.92932
2024-Jun-22 00:00    59.116 19.56331
2024-Jul-23 00:00  61.97734 20.15039
2024-Aug-23 00:00  64.84944 20.68914
2024-Sep-23 00:00  67.73074 21.17832
2024-Oct-24 00:00  70.61942 21.61685
2024-Nov-24 00:00  73.51359 22.00384
2024-Dec-25 00:00  76.41117 22.33857
Length = 260 rows
Monthly intervals based on Jup

STEP 2 - obtaining ScWs

In [5]:
Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj, radius):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw',
            good_isgri = ">1000",
        )
    R.sort('scw_id') 
    return R['scw_id'], R['scw_ver'], R['start_date'], R['end_date'], R['good_isgri']

assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In [ ]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []
good_isgri_list = []

for row in eph:
    ra = float(row['RA'])
    dec = float(row['DEC'])
    radius = 8.
    scw_id, scw_ver, scw_start, scw_end, good_isgri = get_scw_list(ra, dec, radius)
    scw_ids.append(scw_id) # Using append means we get a list of lists, grouped by Jupiter's monthly position
    scw_versions.append(scw_ver)
    scw_start_times.append(scw_start)
    scw_end_times.append(scw_end)
    good_isgri_list.append(good_isgri)

# Filter by type (pointing type)
scw_ids_filtered = []
scw_versions_filtered = []
scw_start_times_filtered = []
scw_end_times_filtered = []
good_isgri_filtered = []

for scw_sublist, ver_sublist, start_sublist, end_sublist, isgri_sublist in zip(scw_ids, scw_versions, scw_start_times, scw_end_times, good_isgri_list):
    
    mask = [int(scw) % 10 == 0 for scw in scw_sublist]
    
    filtered_scws = [scw_sublist[i] for i in range(len(scw_sublist)) if mask[i]]
    filtered_versions = [ver_sublist[i] for i in range(len(ver_sublist)) if mask[i]]
    filtered_start_times = [start_sublist[i] for i in range(len(start_sublist)) if mask[i]]
    filtered_end_times = [end_sublist[i] for i in range(len(end_sublist)) if mask[i]]
    filtered_isgri = [isgri_sublist[i] for i in range(len(isgri_sublist)) if mask[i]]
    
    scw_ids_filtered.append(filtered_scws)
    scw_versions_filtered.append(filtered_versions)
    scw_start_times_filtered.append(filtered_start_times)
    scw_end_times_filtered.append(filtered_end_times)
    good_isgri_filtered.append(filtered_isgri)

scw_ids = scw_ids_filtered
scw_versions = scw_versions_filtered
scw_start_times = scw_start_times_filtered
scw_end_times = scw_end_times_filtered
good_isgri_list = good_isgri_filtered

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids]), " pointing ScWs.")
print("Number of ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids):
    if len(scw_sublist)!=0:
        print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")

Found a total of 408558  pointing ScWs.
Number of ScWs found per month: 

Month 1 - # of ScWs found: 377
Month 2 - # of ScWs found: 358
Month 3 - # of ScWs found: 312
Month 4 - # of ScWs found: 417
Month 5 - # of ScWs found: 354
Month 6 - # of ScWs found: 306
Month 7 - # of ScWs found: 290
Month 8 - # of ScWs found: 302
Month 9 - # of ScWs found: 321
Month 10 - # of ScWs found: 159
Month 11 - # of ScWs found: 147
Month 12 - # of ScWs found: 131
Month 13 - # of ScWs found: 104
Month 14 - # of ScWs found: 95
Month 15 - # of ScWs found: 84
Month 16 - # of ScWs found: 88
Month 17 - # of ScWs found: 96
Month 18 - # of ScWs found: 117
Month 19 - # of ScWs found: 358
Month 20 - # of ScWs found: 966
Month 21 - # of ScWs found: 1665
Month 22 - # of ScWs found: 2386
Month 23 - # of ScWs found: 2958
Month 24 - # of ScWs found: 3136
Month 25 - # of ScWs found: 2847
Month 26 - # of ScWs found: 2278
Month 27 - # of ScWs found: 1746
Month 28 - # of ScWs found: 1135
Month 29 - # of ScWs found: 781
Mon

STEP 3 - filtering by time

In [7]:
scw_ids_filtered = []
scw_versions_filtered = []
scw_start_times_filtered = []
scw_end_times_filtered = []
good_isgri_filtered = []

for month_index, (scw_sublist, ver_sublist, start_sublist, end_sublist, isgri_sublist) in enumerate(zip(scw_ids, scw_versions, scw_start_times, scw_end_times, good_isgri_list)):
    filtered_scws_month = []
    filtered_versions_month = []
    filtered_start_times_month = []
    filtered_end_times_month = []
    filtered_isgri_month = []

    if month_index < len(monthly_intervals):
        month_start, month_end = monthly_intervals[month_index]
    else:
        month_start, month_end = monthly_intervals[-1] 

    for scw_id, start_time, end_time in zip(scw_sublist, start_sublist, end_sublist):
            if month_start <= start_time <= month_end: 
                filtered_scws_month.append(scw_sublist)
                filtered_versions_month.append(ver_sublist[scw_sublist.index(scw_id)])
                filtered_start_times_month.append(start_sublist)
                filtered_end_times_month.append(end_sublist) 
                filtered_isgri_month.append(isgri_sublist)
    
    scw_ids_filtered.append(filtered_scws_month)
    scw_versions_filtered.append(filtered_versions_month)
    scw_start_times_filtered.append(filtered_start_times_month)
    scw_end_times_filtered.append(filtered_end_times_month)
    good_isgri_filtered.append(filtered_isgri_month)

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids_filtered]), "matching ScWs.")
print("Number of matching ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids_filtered):
    if len(scw_sublist)!=0:
        print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")

Found a total of 1741 matching ScWs.
Number of matching ScWs found per month: 

Month 6 - # of ScWs found: 3
Month 24 - # of ScWs found: 234
Month 29 - # of ScWs found: 28
Month 30 - # of ScWs found: 4
Month 43 - # of ScWs found: 1
Month 55 - # of ScWs found: 49
Month 56 - # of ScWs found: 139
Month 57 - # of ScWs found: 27
Month 61 - # of ScWs found: 8
Month 62 - # of ScWs found: 67
Month 68 - # of ScWs found: 6
Month 69 - # of ScWs found: 22
Month 75 - # of ScWs found: 4
Month 81 - # of ScWs found: 6
Month 106 - # of ScWs found: 2
Month 120 - # of ScWs found: 107
Month 121 - # of ScWs found: 20
Month 126 - # of ScWs found: 77
Month 127 - # of ScWs found: 4
Month 158 - # of ScWs found: 4
Month 159 - # of ScWs found: 6
Month 164 - # of ScWs found: 12
Month 165 - # of ScWs found: 40
Month 171 - # of ScWs found: 12
Month 178 - # of ScWs found: 12
Month 179 - # of ScWs found: 13
Month 183 - # of ScWs found: 2
Month 184 - # of ScWs found: 2
Month 191 - # of ScWs found: 57
Month 192 - # of 

Finally, filter for good data (good_isgri>1000).

In [15]:
scw_ids_good = []
scw_versions_good = []
scw_start_times_good = []
scw_end_times_good = []

for scw_sublist, ver_sublist, start_sublist, end_sublist, isgri_sublist in zip(scw_ids_filtered, scw_versions_filtered, scw_start_times_filtered, scw_end_times_filtered, good_isgri_filtered):
    
    mask = [isgri[i] > 1000 for i in range(len(isgri_sublist)) for isgri in isgri_sublist]
    
    filtered_scws = [scw_sublist[i] for i in range(len(scw_sublist)) if mask[i]]
    filtered_versions = [ver_sublist[i] for i in range(len(ver_sublist)) if mask[i]]
    filtered_start_times = [start_sublist[i] for i in range(len(start_sublist)) if mask[i]]
    filtered_end_times = [end_sublist[i] for i in range(len(end_sublist)) if mask[i]]
    
    scw_ids_good.append(filtered_scws)
    scw_versions_good.append(filtered_versions)
    scw_start_times_good.append(filtered_start_times)
    scw_end_times_good.append(filtered_end_times)

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids_good]), "good ScWs.")
print("Number of ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids_good):
    if len(scw_sublist)!=0:
        print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")

Found a total of 812 good ScWs.
Number of ScWs found per month: 

Month 29 - # of ScWs found: 28
Month 57 - # of ScWs found: 27
Month 61 - # of ScWs found: 8
Month 62 - # of ScWs found: 67
Month 81 - # of ScWs found: 6
Month 159 - # of ScWs found: 6
Month 191 - # of ScWs found: 57
Month 197 - # of ScWs found: 120
Month 198 - # of ScWs found: 276
Month 199 - # of ScWs found: 154
Month 203 - # of ScWs found: 41
Month 204 - # of ScWs found: 13
Month 222 - # of ScWs found: 2
Month 223 - # of ScWs found: 3
Month 230 - # of ScWs found: 2
Month 231 - # of ScWs found: 2


We can save these ScWs to a file, with Jupiter's expected position.

In [17]:
# with open(f"{epochs['start']}_{epochs['stop']}.txt", "w+") as f:
    # for (ids, vers, starts, ends) in zip(scw_ids, scw_versions, scw_start_times, scw_end_times):
        # f.write("{0}, {1}, {2}, {3}\n".format(ids, vers, starts, ends))

# Flatten the lists
flat_ids = [item for sublist in scw_ids_good for item in sublist]
flat_versions = [item for sublist in scw_versions_good for item in sublist]
flat_start_times = [item for sublist in scw_start_times_good for item in sublist]
flat_end_times = [item for sublist in scw_end_times_good for item in sublist]

with open(f"../data/{epochs['start']}_{epochs['stop']}.txt", "w") as f:
    f.write("ID, Version, Start Time, End Time, Jupiter RA, Jupiter DEC\n")
    for id, version, start, end, ra, dec in zip(flat_ids, flat_versions, flat_start_times, flat_end_times, ra_values, dec_values):
        f.write(f"{id}, {version}, {start}, {end}, {ra}, {dec}\n")

## CRAB

In [19]:
source_name = 'Crab'

Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj, radius):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw',
            good_isgri = ">1000",
        )
    R.sort('scw_id') 
    # R['ra', 'dec'].pprint()
    return R['scw_id'], R['scw_ver'], R['start_date'], R['end_date'], R['good_isgri']

assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

result_table = Simbad.query_object(source_name)
print(result_table)
source_coord = coord.SkyCoord(result_table['ra'][0], result_table['dec'][0], unit=("deg", "deg"))
ra=source_coord.ra.deg
dec=source_coord.dec.deg
radius=8.

scwlist,version,start_all,end_all,isgri_all = get_scw_list(ra,dec,radius)

main_id    ra     dec   ... coo_wavelength     coo_bibcode     matched_id
          deg     deg   ...                                              
------- ------- ------- ... -------------- ------------------- ----------
  M   1 83.6324 22.0174 ...              X 2022A&A...661A..38P  NAME Crab


Filter for pointing ScWs.

In [20]:
scw = [(s+"."+v).strip() for s,v in zip(scwlist,version) if s.endswith('0')]
start = [t for s, t in zip(scwlist, start_all) if s.endswith('0')]
end = [t for s, t in zip(scwlist, end_all) if s.endswith('0')]
isgri = [i for s, i in zip(scwlist, isgri_all) if s.endswith('0')]

print(f"Found {len(scw)} Crab ScWs of type pointing.")

Found 8733 Crab ScWs of type pointing.


Filter for good data.

In [21]:
isgri_mask = [i>1000 for i in isgri]

scw = [scw[i] for i in range(len(scw)) if isgri_mask[i]]
start = [start[i] for i in range(len(start)) if isgri_mask[i]]
end = [end[i] for i in range(len(end)) if isgri_mask[i]]

print(f"Found {len(scw)} Crab ScWs of type pointing and with good_isgri > 1000.")

Found 7769 Crab ScWs of type pointing and with good_isgri > 1000.


Save to file.

In [22]:
with open(f"../data/Crab-ScWs.txt", "w") as f:
    f.write("SCW, Start Time, End Time\n")
    for s, start, end in zip(scw, start, end):
        f.write(f"{s}, {start}, {end}\n")